In [ ]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

url = 'https://github.com/millenopan/Cardiovascular-Health-Project/blob/main/processed-cleveland.csv?raw=true'
hd = pd.read_csv(url)
hd.target = hd.target.replace({2:1, 3:1, 4:1})
url = 'https://github.com/millenopan/Cardiovascular-Health-Project/blob/main/heart-failure.csv?raw=true'
hf = pd.read_csv(url)
from sklearn.metrics import confusion_matrix, accuracy_score, plot_roc_curve, classification_report

In [ ]:
hf

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [ ]:
x = hf.drop(['DEATH_EVENT'], axis = 1)
y = hf.DEATH_EVENT
training_x, testing_x, training_y, testing_y = train_test_split(x,y, test_size=0.2, random_state=85)
svc = SVC()
svc.fit(training_x, training_y)
svc_training_pred = svc.predict(training_x)
svc_testing_pred = svc.predict(testing_x)

pca = PCA(n_components=4)
training_x_pca = pca.fit_transform(training_x)
testing_x_pca = pca.transform(testing_x)

explained_variance = pca.explained_variance_ratio_
print(explained_variance)

svc_confusion_mat = confusion_matrix(testing_y, svc_testing_pred)
svc_accuracy = accuracy_score(testing_y, svc_testing_pred)
print("Confusion matrix:")
print(svc_confusion_mat)
print("Accuracy: " + str(svc_accuracy))

report = classification_report(testing_y, svc_testing_pred)
print("Report of this classification")
print(report)

[9.99893394e-01 1.05931792e-04 6.44290933e-07 1.58194606e-08]
Confusion matrix:
[[46  0]
 [14  0]]
Accuracy: 0.7666666666666667
Report of this classification
              precision    recall  f1-score   support

           0       0.77      1.00      0.87        46
           1       0.00      0.00      0.00        14

    accuracy                           0.77        60
   macro avg       0.38      0.50      0.43        60
weighted avg       0.59      0.77      0.67        60



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The accuracy without any modifications to the data is 76.7 percent. Next, we can try fitting the data to a Gradient Boosting classifier.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(training_x, training_y)
gb_training = gbc.predict(training_x)
gb_testing = gbc.predict(testing_x)

scores = cross_val_score(gbc, x, y, cv=40)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores

Accuracy: 0.80 (+/- 0.28)


array([0.375     , 0.375     , 0.625     , 0.75      , 0.875     ,
       1.        , 0.75      , 0.875     , 0.875     , 0.875     ,
       0.875     , 0.875     , 0.875     , 0.75      , 1.        ,
       0.875     , 0.875     , 1.        , 0.875     , 0.71428571,
       0.71428571, 1.        , 0.85714286, 0.71428571, 0.71428571,
       0.85714286, 0.85714286, 0.85714286, 0.85714286, 0.85714286,
       0.57142857, 0.85714286, 0.71428571, 0.85714286, 0.71428571,
       0.85714286, 0.71428571, 0.85714286, 0.71428571, 0.85714286])

The accuracy is a little higher. But we'll now try logistic regression and see what that yields. 

In [ ]:
x = hf.drop(['DEATH_EVENT'], axis = 1)
y = hf.DEATH_EVENT
training_x, testing_x , training_y, testing_y = train_test_split(x, y, test_size=0.2, random_state=85)
lr = LogisticRegression(max_iter=2000)
lr.fit(training_x, training_y)
lr_train_prediction = lr.predict(training_x)
lr_test_prediction = lr.predict(testing_x)
print("Logistical regression accuracy:")
lr.score(testing_x, testing_y)

confusion_mat = confusion_matrix(testing_y, lr_test_prediction)
accuracy = accuracy_score(testing_y, lr_test_prediction)
print("Confusion matrix:")
confusion_mat
print("Accuracy: " + str(accuracy))

report = classification_report(testing_y, lr_test_prediction)
print("Report of this classification")
print(report)

Logistical regression accuracy:
Confusion matrix:
Accuracy: 0.8333333333333334
Report of this classification
              precision    recall  f1-score   support

           0       0.91      0.87      0.89        46
           1       0.62      0.71      0.67        14

    accuracy                           0.83        60
   macro avg       0.77      0.79      0.78        60
weighted avg       0.84      0.83      0.84        60



The accuracy is 83.3 percent. Next, we will try increasing the test size to 0.3 and I will also change the fit intercept to false, set verbose to 1, and n_jobs (number of CPU scores used) to a nonzero number. As well as this, I'll try to find the ideal maximum iterations value. 

In [ ]:
x = hf.drop(['DEATH_EVENT'], axis = 1)
y = hf.DEATH_EVENT
training_x, testing_x , training_y, testing_y = train_test_split(x, y, test_size=0.3, random_state=85)
lr = LogisticRegression(fit_intercept=False, max_iter=70, verbose = 1, n_jobs = 8)
lr.fit(training_x, training_y)
lr_train_prediction = lr.predict(training_x)
lr_test_prediction = lr.predict(testing_x)
print("Logistical regression accuracy:")
print(lr.score(testing_x, testing_y))

confusion_mat = confusion_matrix(testing_y, lr_test_prediction)
accuracy = accuracy_score(testing_y, lr_test_prediction)
print("Confusion matrix:")
print(confusion_mat)
print("Accuracy: " + str(accuracy))

report = classification_report(testing_y, lr_test_prediction)
print("Report of this classification:")
print(report)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Logistical regression accuracy:
0.8777777777777778
Confusion matrix:
[[58  7]
 [ 4 21]]
Accuracy: 0.8777777777777778
Report of this classification:
              precision    recall  f1-score   support

           0       0.94      0.89      0.91        65
           1       0.75      0.84      0.79        25

    accuracy                           0.88        90
   macro avg       0.84      0.87      0.85        90
weighted avg       0.88      0.88      0.88        90



[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:    2.0s finished


The best maximum iterations value (which leads to the highest accuracy) looks to be 70. This leads to an accuracy of 87.8 percent (to 3 signiificant figures), which is a big improvement from earlier. 